# Improved RLV Descent Simulation with Corrected Physics

## Improvements:
1. **Proper atmospheric model** - Exponential density decay with altitude
2. **Complete 6-DOF dynamics** - Position, velocity, attitude, angular rates
3. **Realistic physics** - Mass, moment of inertia, gravity, drag
4. **Performance optimization** - Pre-allocated arrays, vectorized operations
5. **Fixed simulation** - Actually reaches ground level
6. **Proper control** - Attitude control instead of altitude control

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

# Physical Constants
G = 9.81  # Gravity (m/s^2)
RHO_0 = 1.225  # Sea level air density (kg/m^3)
H_SCALE = 8500  # Atmospheric scale height (m)
VEHICLE_MASS = 25000  # Vehicle mass (kg)
MOMENT_INERTIA = np.array([50000, 50000, 10000])  # Moment of inertia [Ixx, Iyy, Izz] (kg·m^2)
CROSS_SECTION_AREA = 10  # Cross-sectional area (m^2)
DRAG_COEFFICIENT = 0.5  # Drag coefficient

# Grid Fin Parameters
FIN_AREA = 0.5  # Grid fin area (m^2)
LIFT_COEFFICIENT = 1.2  # Lift coefficient
FIN_DRAG_COEFFICIENT = 0.3  # Fin drag coefficient

# RCS Parameters
THRUSTER_FORCE = 5000  # Thrust per thruster (N)
THRUSTER_ARM = 2.5  # Distance from CoM to thruster (m)
FUEL_CONSUMPTION_RATE = 0.1  # kg/s per thruster

# Simulation Parameters
T_FINAL = 1000  # Maximum simulation time (s)
DT = 0.1  # Time step (s)
NUM_STEPS = int(T_FINAL / DT)

# Initial Conditions
ALTITUDE_0 = 40000  # Initial altitude (m)
VELOCITY_0 = -50  # Initial vertical velocity (m/s, negative = downward)

# Control Parameters
ALTITUDE_RCS = 30000  # Altitude threshold for RCS mode (m)
ALTITUDE_GRID = 10000  # Altitude threshold for Grid Fins mode (m)
PERTURBATION_INTERVAL = 50  # Steps between perturbations
PERTURBATION_MAG = 0.1  # Perturbation magnitude (rad/s)

print("Constants and parameters initialized successfully.")

In [ ]:
# Helper Functions

def atmospheric_density(altitude):
    """Calculate air density using exponential atmospheric model."""
    return RHO_0 * np.exp(-altitude / H_SCALE)

def control_mode_selector(altitude):
    """Select control mode based on altitude."""
    if altitude > ALTITUDE_RCS:
        return "RCS"
    elif altitude > ALTITUDE_GRID:
        return "Hybrid"
    else:
        return "Grid Fins"

def rcs_torque(angular_velocity, desired_angular_velocity):
    """
    Calculate RCS torque based on angular velocity error.
    Uses thruster pairs to generate corrective torque.
    """
    error = desired_angular_velocity - angular_velocity
    # Proportional control for torque generation
    control_gain = 10000  # Nm/(rad/s)
    torque = control_gain * error
    # Limit to available thruster torque
    max_torque = THRUSTER_FORCE * THRUSTER_ARM
    return np.clip(torque, -max_torque, max_torque)

def aerodynamic_torque(velocity, angular_velocity, air_density, angle_of_attack):
    """
    Calculate torque from grid fins based on aerodynamic forces.
    """
    v_mag = np.abs(velocity)
    if v_mag < 1:
        return np.zeros(3)
    
    # Aerodynamic moment (simplified model)
    q = 0.5 * air_density * v_mag**2  # Dynamic pressure
    
    # Restoring torque proportional to angle of attack
    torque_magnitude = q * FIN_AREA * THRUSTER_ARM * np.sin(angle_of_attack)
    
    # Damping torque opposing angular velocity
    damping = -0.5 * air_density * v_mag * FIN_AREA * THRUSTER_ARM**2 * angular_velocity
    
    return np.array([torque_magnitude, 0, 0]) + damping

def hybrid_torque(rcs_t, aero_t, altitude):
    """
    Blend RCS and aerodynamic torques based on altitude.
    """
    # Linear interpolation between ALTITUDE_GRID and ALTITUDE_RCS
    weight_rcs = (altitude - ALTITUDE_GRID) / (ALTITUDE_RCS - ALTITUDE_GRID)
    weight_rcs = np.clip(weight_rcs, 0, 1)
    weight_aero = 1 - weight_rcs
    return weight_rcs * rcs_t + weight_aero * aero_t

def drag_force(velocity, air_density):
    """Calculate aerodynamic drag force."""
    return 0.5 * DRAG_COEFFICIENT * air_density * CROSS_SECTION_AREA * velocity * np.abs(velocity)

print("Helper functions defined.")

In [ ]:
# PID Controller Class (improved)
class PIDController:
    def __init__(self, kp, ki, kd, integral_limit=100):
        self.kp = kp
        self.ki = ki
        self.kd = kd
        self.integral_limit = integral_limit
        self.previous_error = 0
        self.integral = 0

    def compute(self, error, dt):
        # Anti-windup: limit integral term
        self.integral += error * dt
        self.integral = np.clip(self.integral, -self.integral_limit, self.integral_limit)
        
        derivative = (error - self.previous_error) / dt if dt > 0 else 0
        self.previous_error = error
        
        return self.kp * error + self.ki * self.integral + self.kd * derivative
    
    def reset(self):
        self.integral = 0
        self.previous_error = 0

# PID controllers for 3-axis attitude control
pid_roll = PIDController(kp=0.5, ki=0.05, kd=0.2)
pid_pitch = PIDController(kp=0.5, ki=0.05, kd=0.2)
pid_yaw = PIDController(kp=0.3, ki=0.02, kd=0.1)

print("PID controllers initialized.")

In [ ]:
# Pre-allocate arrays for performance
time = np.zeros(NUM_STEPS)
altitude = np.zeros(NUM_STEPS)
velocity = np.zeros(NUM_STEPS)
angular_velocity = np.zeros((NUM_STEPS, 3))
attitude = np.zeros((NUM_STEPS, 3))  # Euler angles [roll, pitch, yaw]
lateral_position = np.zeros((NUM_STEPS, 2))  # [x, y] position
fuel_used = np.zeros(NUM_STEPS)
torque_magnitude = np.zeros(NUM_STEPS)
control_mode = np.empty(NUM_STEPS, dtype=object)
air_density_array = np.zeros(NUM_STEPS)

# Initial conditions
altitude[0] = ALTITUDE_0
velocity[0] = VELOCITY_0
angular_velocity[0] = np.array([0.0, 0.0, 0.0])
attitude[0] = np.array([0.0, 0.0, 0.0])
lateral_position[0] = np.array([0.0, 0.0])
fuel_used[0] = 0.0
control_mode[0] = control_mode_selector(altitude[0])

print("State arrays pre-allocated.")

In [ ]:
# Main Simulation Loop
print("Starting simulation...")

for i in range(NUM_STEPS - 1):
    time[i+1] = time[i] + DT
    
    # Current state
    h = altitude[i]
    v = velocity[i]
    omega = angular_velocity[i]
    theta = attitude[i]
    pos_lateral = lateral_position[i]
    
    # Stop if reached ground
    if h <= 0:
        # Fill remaining arrays
        altitude[i+1:] = 0
        velocity[i+1:] = 0
        angular_velocity[i+1:] = angular_velocity[i]
        attitude[i+1:] = attitude[i]
        lateral_position[i+1:] = lateral_position[i]
        fuel_used[i+1:] = fuel_used[i]
        control_mode[i+1:] = "Ground"
        print(f"Landed at t={time[i]:.2f}s")
        break
    
    # Atmospheric density
    rho = atmospheric_density(h)
    air_density_array[i] = rho
    
    # Select control mode
    mode = control_mode_selector(h)
    control_mode[i+1] = mode
    
    # Desired attitude: maintain vertical (roll=0, pitch=0, yaw=0)
    desired_attitude = np.array([0.0, 0.0, 0.0])
    desired_omega = np.array([0.0, 0.0, 0.0])
    
    # Apply perturbations
    if i % PERTURBATION_INTERVAL == 0 and i > 0:
        omega += PERTURBATION_MAG * np.random.randn(3)
    
    # Compute attitude error for PID
    attitude_error = desired_attitude - theta
    omega_error = desired_omega - omega
    
    # Calculate control torque based on mode
    if mode == "RCS":
        # Pure RCS control
        torque = rcs_torque(omega, desired_omega)
        fuel_rate = np.linalg.norm(torque) / (THRUSTER_FORCE * THRUSTER_ARM) * FUEL_CONSUMPTION_RATE
        
    elif mode == "Grid Fins":
        # Pure aerodynamic control
        torque = aerodynamic_torque(v, omega, rho, theta[0])  # Using roll angle
        fuel_rate = 0  # No fuel consumption for grid fins
        
    else:  # Hybrid
        # Blended control
        rcs_t = rcs_torque(omega, desired_omega)
        aero_t = aerodynamic_torque(v, omega, rho, theta[0])
        torque = hybrid_torque(rcs_t, aero_t, h)
        # Fuel consumption proportional to RCS usage
        weight_rcs = (h - ALTITUDE_GRID) / (ALTITUDE_RCS - ALTITUDE_GRID)
        fuel_rate = weight_rcs * np.linalg.norm(rcs_t) / (THRUSTER_FORCE * THRUSTER_ARM) * FUEL_CONSUMPTION_RATE
    
    # Dynamics integration
    # Angular acceleration: alpha = I^-1 * torque
    alpha = torque / MOMENT_INERTIA
    
    # Update angular velocity
    omega_new = omega + alpha * DT
    
    # Update attitude (Euler integration)
    theta_new = theta + omega * DT
    
    # Linear dynamics
    # Drag force (opposes velocity)
    F_drag = -drag_force(v, rho)
    
    # Gravity force
    F_gravity = -VEHICLE_MASS * G
    
    # Net force and acceleration
    F_net = F_gravity + F_drag
    a = F_net / VEHICLE_MASS
    
    # Update velocity and altitude
    v_new = v + a * DT
    h_new = h + v * DT
    
    # Lateral position (simplified: small angle approximation)
    # Lateral velocity due to tilt
    v_lateral = np.array([v * np.sin(theta[1]), v * np.sin(theta[0])])
    pos_lateral_new = pos_lateral + v_lateral * DT
    
    # Update state
    altitude[i+1] = max(h_new, 0)
    velocity[i+1] = v_new
    angular_velocity[i+1] = omega_new
    attitude[i+1] = theta_new
    lateral_position[i+1] = pos_lateral_new
    fuel_used[i+1] = fuel_used[i] + fuel_rate * DT
    torque_magnitude[i] = np.linalg.norm(torque)
    
    # Progress update
    if (i+1) % 1000 == 0:
        print(f"Step {i+1}/{NUM_STEPS}: Alt={h:.0f}m, Vel={v:.1f}m/s, Mode={mode}")

print("\nSimulation Complete!")
print(f"Final Altitude: {altitude[i+1]:.2f} m")
print(f"Total Fuel Used: {fuel_used[i+1]:.2f} kg")
print(f"Final Lateral Deviation: {np.linalg.norm(lateral_position[i+1]):.2f} m")

In [ ]:
# Advanced Visualization
fig, axes = plt.subplots(3, 2, figsize=(16, 12))
fig.suptitle('RLV Descent Simulation - Corrected Physics Model', fontsize=16, fontweight='bold')

# Define mode colors
mode_colors = {'RCS': '#ff6b6b', 'Hybrid': '#ffa500', 'Grid Fins': '#4ecdc4', 'Ground': '#95a5a6'}

# Helper function to color by mode
def plot_by_mode(ax, x, y, ylabel, title):
    for mode_name, color in mode_colors.items():
        mask = control_mode == mode_name
        if np.any(mask):
            ax.plot(x[mask], y[mask], color=color, label=mode_name, linewidth=2, alpha=0.8)
    ax.set_xlabel('Time (s)', fontsize=10)
    ax.set_ylabel(ylabel, fontsize=10)
    ax.set_title(title, fontsize=11, fontweight='bold')
    ax.grid(True, alpha=0.3)
    ax.legend(loc='best', fontsize=9)

# Plot 1: Altitude vs Time
plot_by_mode(axes[0,0], time, altitude, 'Altitude (m)', 'Altitude Profile')

# Plot 2: Velocity vs Time
plot_by_mode(axes[0,1], time, velocity, 'Velocity (m/s)', 'Descent Velocity')

# Plot 3: Fuel Consumption
plot_by_mode(axes[1,0], time, fuel_used, 'Fuel Used (kg)', 'Cumulative Fuel Consumption')

# Plot 4: Torque Magnitude
plot_by_mode(axes[1,1], time, torque_magnitude, 'Torque (Nm)', 'Control Torque Magnitude')

# Plot 5: Angular Velocity
axes[2,0].plot(time, angular_velocity[:, 0], 'r-', label='Roll rate', linewidth=1.5)
axes[2,0].plot(time, angular_velocity[:, 1], 'g-', label='Pitch rate', linewidth=1.5)
axes[2,0].plot(time, angular_velocity[:, 2], 'b-', label='Yaw rate', linewidth=1.5)
axes[2,0].set_xlabel('Time (s)', fontsize=10)
axes[2,0].set_ylabel('Angular Velocity (rad/s)', fontsize=10)
axes[2,0].set_title('Angular Velocity Components', fontsize=11, fontweight='bold')
axes[2,0].grid(True, alpha=0.3)
axes[2,0].legend(loc='best', fontsize=9)

# Plot 6: Lateral Deviation
lateral_deviation = np.linalg.norm(lateral_position, axis=1)
plot_by_mode(axes[2,1], time, lateral_deviation, 'Lateral Deviation (m)', 'Trajectory Deviation from Vertical')

plt.tight_layout()
plt.savefig('rlv_simulation_corrected.png', dpi=300, bbox_inches='tight')
plt.show()

print("Plots saved as 'rlv_simulation_corrected.png'")

In [ ]:
# Additional Analysis: Mode Comparison
print("\n=== Performance Analysis by Control Mode ===")
print()

for mode_name in ['RCS', 'Hybrid', 'Grid Fins']:
    mask = control_mode == mode_name
    if not np.any(mask):
        continue
    
    mode_indices = np.where(mask)[0]
    start_idx = mode_indices[0]
    end_idx = mode_indices[-1]
    
    alt_range = f"{altitude[start_idx]:.0f}m to {altitude[end_idx]:.0f}m"
    time_duration = time[end_idx] - time[start_idx]
    fuel_consumed = fuel_used[end_idx] - fuel_used[start_idx]
    avg_torque = np.mean(torque_magnitude[mask])
    max_angular_vel = np.max(np.linalg.norm(angular_velocity[mask], axis=1))
    
    print(f"{mode_name} Mode:")
    print(f"  Altitude Range: {alt_range}")
    print(f"  Duration: {time_duration:.1f}s")
    print(f"  Fuel Consumed: {fuel_consumed:.2f}kg")
    print(f"  Average Torque: {avg_torque:.2f}Nm")
    print(f"  Max Angular Velocity: {max_angular_vel:.4f}rad/s")
    print()

# Overall statistics
print("=== Overall Statistics ===")
print(f"Total Mission Duration: {time[i+1]:.1f}s ({time[i+1]/60:.2f} minutes)")
print(f"Average Descent Rate: {ALTITUDE_0/time[i+1]:.2f}m/s")
print(f"Maximum Velocity: {np.min(velocity):.2f}m/s")
print(f"Final Lateral Deviation: {lateral_deviation[i+1]:.2f}m")
print(f"Fuel Efficiency: {ALTITUDE_0/fuel_used[i+1]:.2f}m/kg")